In [3]:
import numpy as np

'''
for every g in generations // loopy version, inefficient v2
	for every N in Ndim
		for every cell in N // do this for every cell
			for every N in Ndims // get the neighbourhood of cell
				get neighbourhood cells to cell for every N in Ndims // going to be 3 per N dim = 3 * Ndims
			for every cell in neighbourhood
				get prev cell state for every cell in neighbourhood // state is vector of activations
					if no cell state
						cell state is zero_state
			concat prev cell states to one neighbourhood state // neighbourhood is Ndim matrix of state vectors
			get new cell state from neighbourhood states and input // for next generation

'''

class Cell(object):
    def __init__(self):
        pass
    def __call__(self, in_, state):
        return 1


    # 1d
batch = 1
window = 3

a = np.arange(9).reshape((batch, 9))
print (a[0], '\n')

for g in range(1):
    
    neighbourhoods = []
    for i in range(len(a[0])):
        index = i + 2
        neighbourhood = a[0][max(0, index - window):index]
        for n, neighbour in enumerate(neighbourhood):
            # print ('neighbour', neighbour)
            if not neighbour:  # if no state
                neighbourhood[n] = 0;  # get zero_state
        
        # do something with neighbourhood
        # calculate new cell state
        print(np.sum(neighbourhood))
        print ('neighbourhood', i, neighbourhood)
        
        # print ('neighbourhood', neighbourhood)
#         neighbourhoods.append(neighbourhood)
        
#     neighbourhoods = np.concatenate(neighbourhoods, axis=1)
#     print ('neighbourhoods', neighbourhoods)
    
#         state = np.sum(neighbourhood, axis=0)  # concat
#         print ('cell {} with neighours {}'.format(c, neighbourhood))
#         print (state)
    
#     for N in range(len(a.shape)):
#         print (N)
#         for cell in range(a.shape[N]):
#             print('cell {} in dim {}'.format(cell, N))
#             neighbourhood = np.zeros([3] * len(a.shape))
#             for n in range(len(neighbourhood.shape)):
#                 for i in range(neighbourhood.shape[n]):
#                     neighbourhood[n][i] = 1

#             print (neighbourhood)

[0 1 2 3 4 5 6 7 8] 

1
neighbourhood 0 [0 1]
3
neighbourhood 1 [0 1 2]
6
neighbourhood 2 [1 2 3]
9
neighbourhood 3 [2 3 4]
12
neighbourhood 4 [3 4 5]
15
neighbourhood 5 [4 5 6]
18
neighbourhood 6 [5 6 7]
21
neighbourhood 7 [6 7 8]
15
neighbourhood 8 [7 8]
